In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(4, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Source(model,'Source')
node[3] = Sink(model,'Sink')

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 2, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

node[0].set_service(jobclass[0], Erlang(3,2))
node[0].set_service(jobclass[1], HyperExp(0.5,3,10))

node[1].set_service(jobclass[0], HyperExp(0.1,1,10))
node[1].set_service(jobclass[1], Exp(1))

node[2].set_arrival(jobclass[1], Exp(0.1))

M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Delay <-> Queue1 (circul(2))
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Sink
P.set(jobclass[1], jobclass[1], node[2], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[3], 1.0)
            
model.link(P)

In [ ]:
print('This example shows the execution of the solver on a 2-class 2-node mixed model.')

# Configure solver options
options = Solver.default_options()
options['keep'] = True
options['verbose'] = 1
options['cutoff'] = 3
options['seed'] = 23000

# Create multiple solvers for comparison
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model, cutoff=3))  # CTMC is infinite on this model
solver = np.append(solver, JMT(model, seed=23000, verbose=True, keep=False))
solver = np.append(solver, SSA(model, seed=23000, verbose=False, samples=10000))
solver = np.append(solver, FLD(model))
solver = np.append(solver, MVA(model))
solver = np.append(solver, NC(model))
solver = np.append(solver, MAM(model))
solver = np.append(solver, DES(model, seed=23000))

# Solve and display results
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    try:
        print(f'\nSOLVER: {solver[s].get_name()}')
        AvgTable[s] = solver[s].avg_table()
        print(AvgTable[s])
    except Exception as e:
        print(f'Error with {solver[s].get_name()}: {e}')